# LangGraph Agent v2 — Search + Process

Adds web search and synthesis (no enrichment step).

Quick guide:
- State: messages, search_results, optimized_query
- Nodes: analyze → search → process → END
- Observe prints: [ANALYZE], [SEARCH], [PROCESS]
- Run the last cell to see a clean, single execution trace

In [11]:
import os
from typing import List, Optional, Union, Literal
from dotenv import load_dotenv
from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, END

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

from typing import Annotated
from langgraph.graph import add_messages

from langchain_openai import ChatOpenAI
from langchain_community.utilities import GoogleSearchAPIWrapper
load_dotenv()

True

In [12]:
class SearchResult(BaseModel):
    id: int
    title: str
    snippet: str
    link: str

class AgentState(BaseModel):
    messages: Annotated[List[Union[HumanMessage, AIMessage, SystemMessage]], add_messages] = Field(description="The chat history")
    search_results: List[SearchResult] = Field(default_factory=list, description="The results from the web search")
    optimized_query: Optional[str] = Field(default=None, description="The LLM-optimized search query")
    
    class Config:
        arbitrary_types_allowed = True

In [13]:


llm = ChatOpenAI(
  base_url="https://api.mistral.ai/v1",
  api_key=os.environ["MISTRAL_API_KEY"],
  model="mistral-large-2512",
)

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)

search_engine = GoogleSearchAPIWrapper()


In [14]:
def analyze_query(state: AgentState):
    print("[ANALYZE] Optimizing user query…")
    user_query = next((m.content for m in reversed(state.messages) if isinstance(m, HumanMessage)), "")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You rewrite user queries into high-recall google web search queries. Today's date is 2025-11-05. Preserve key entities and constraints (dates, names, locations). Add helpful synonyms if useful. Return ONLY the optimized query with no quotes or extra text."),
        ("human", "Optimize: {q}")
    ])
    q = llm.invoke(prompt.format_messages(q=user_query)).content.strip().replace('\"','')
    return {"optimized_query": q}

In [15]:
def perform_search(state: AgentState):
    print("[SEARCH] Running web search…")
    query = state.optimized_query or next((m.content for m in reversed(state.messages) if isinstance(m, HumanMessage)), "")
    try:
        raw = search_engine.results(query, num_results=5)
    except Exception:
        raw = [{"title":"Search unavailable","snippet":f"Could not search: {query}","link":"https://example.com"}]
    results = [SearchResult(id=i+1,title=r.get('title','No title'),snippet=r.get('snippet',r.get('description','No content')),link=r.get('link',r.get('url','No link'))) for i,r in enumerate(raw)]
    return {"search_results": results}

In [16]:
def process_results(state: AgentState):
    print("[PROCESS] Synthesizing final answer…")
    user_query = next((m.content for m in state.messages if isinstance(m, HumanMessage)), "Unknown query")
    if state.search_results:
        blocks = []
        for r in state.search_results:
            blocks.append(f"Result {r.id} | {r.title}\n{r.snippet}\nSource: {r.link}")
        results_block = "\n".join(blocks)
    else:
        results_block = "No search results available."
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You have to use the provided search results to answer. Be concise and precise. Cite sources (URLs) inline."),
        ("human", "Query: {q}\n\nResults:\n{res}")
    ])
    ans = llm.invoke(prompt.format_messages(q=user_query, res=results_block)).content.strip()
    return {"messages": [AIMessage(content=ans)]}

In [17]:
workflow_v2 = StateGraph(AgentState)
workflow_v2.add_node('analyze', analyze_query)
workflow_v2.add_node('search', perform_search)
workflow_v2.add_node('process', process_results)


workflow_v2.set_entry_point('analyze')

workflow_v2.add_edge('analyze','search')
workflow_v2.add_edge('search','process')
workflow_v2.add_edge('process',END)

agent_v2 = workflow_v2.compile()



In [18]:
# Test v2
query = "What teams were in 2025 NBA playoffs?"
state = AgentState(messages=[HumanMessage(content=query)])

final_state = agent_v2.invoke(state)



[ANALYZE] Optimizing user query…
[SEARCH] Running web search…
[PROCESS] Synthesizing final answer…


In [19]:
for m in final_state['messages']:
    if isinstance(m, HumanMessage):
        print(f'Human: {m.content}')
    elif isinstance(m, AIMessage):
        print(f'AI: {m.content}')

Human: What teams were in 2025 NBA playoffs?
AI: The provided search results indicate that the 2025 NBA playoffs were the postseason tournament for the 2024–25 NBA season (Source 4). While a comprehensive list of all participating teams is not available in the provided information, the Oklahoma City Thunder are mentioned in connection with these playoffs (Source 4).


In [ ]:
final_state

{'messages': [HumanMessage(content='What teams were in 2025 NBA playoffs?', additional_kwargs={}, response_metadata={}, id='10ea39b1-146e-41f6-81e0-e94441fb71a6'),
  AIMessage(content='The provided search results indicate that the 2025 NBA playoffs were the postseason tournament for the 2024–25 NBA season (Source 4). While a comprehensive list of all participating teams is not available in the provided information, the Oklahoma City Thunder are mentioned in connection with these playoffs (Source 4).', additional_kwargs={}, response_metadata={}, id='0cbc7cd1-584c-4893-952c-d6bfa9aa0a07')],
 'search_results': [SearchResult(id=1, title='2025 NBA Playoffs | Home | NBA.com', snippet='List of NBA champions · Every NBA Finals winner in history. Take a look back at every team to win an NBA championship in league history. NBA Organization. NBA\xa0...', link='https://www.nba.com/playoffs/2025'),
  SearchResult(id=2, title='2024-25 NBA Playoffs Bracket - ESPN', snippet="Teams · Trade Machine · Od

: 